# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
!pip install python-dotenv


In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [5]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [6]:
import pandas as pd

# Define the tables and their descriptions
data = {
    'table': ['employees', 'salary', 'studies'],
    'definition': [
        'Employee information, name, position, department, and start date.',
        'Salary details for each year, including base, bonus, and adjustments.',
        'Educational studies, name of the institution, degree, and graduation year.'
    ]
}

df = pd.DataFrame(data)
print(df)

       table                                         definition
0  employees  Employee information, name, position, departme...
1     salary  Salary details for each year, including base, ...
2    studies  Educational studies, name of the institution, ...


In [7]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [8]:
print(text_tables)

employees: Employee information, name, position, department, and start date.
salary: Salary details for each year, including base, bonus, and adjustments.
studies: Educational studies, name of the institution, degree, and graduation year.


In [9]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [11]:
pqt1 = prompt_question_tables.format(tables=text_tables, question= "Which employees have a master's degree and started after 2015?")#ENTER YOUR QUERY HERE)


In [12]:
print(return_OAI(pqt1))

{
    "tables": ["employees", "studies"]
}


In [14]:
prompt_question_tables = """
Given the following tables and their full definitions:

### Tables
{tables}

From the user's question below, identify which tables are needed to write the SQL query.

Return your answer as a JSON object like this:
{{
    "tables": {{
        "table_name": "short description or label from the full definition"
    }}
}}

Only include relevant tables and use a short title based on each table's description.

### User Question:
{question}
"""

In [16]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="Find employees who earned a bonus in 2022 and also have a postgraduate degree.")#ENTER YOUR QUERY HERE)

In [17]:
print(return_OAI(pqt3))

{
    "tables": {
        "employees": "Employee information",
        "salary": "Salary details for each year",
        "studies": "Educational studies"
    }
}


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [19]:
prompt_question_tables = """
Given the following tables and their full definitions:

### Tables
{tables}

From the user's question below, identify which tables are needed to write the SQL query.

Return your answer as a JSON object like this:
{{
    "tables": {{
        "table_name": "short description or label from the full definition"
    }}
}}

Only include relevant tables and use a short title based on each table's description.

### User Question:
{question}
"""

# Prompt A
pqt4 = prompt_question_tables.format(
    tables=text_tables,
    question="Which employees have worked in more than one department since joining the company?"
)
print(return_OAI(pqt4))

# Prompt B
pqt5 = prompt_question_tables.format(
    tables=text_tables,
    question="List employees who started working the same year they graduated."
)
print(return_OAI(pqt5))

# Prompt C
pqt6 = prompt_question_tables.format(
    tables=text_tables,
    question="Which employees hold only a bachelor's degree and have never received a salary increase?"
)
print(return_OAI(pqt6))

# Prompt D
pqt7 = prompt_question_tables.format(
    tables=text_tables,
    question="What are the most common academic degrees among employees who received bonuses?"
)
print(return_OAI(pqt7))

# Prompt E
pqt8 = prompt_question_tables.format(
    tables=text_tables,
    question="How many employees hold more than one academic degree and currently work in the HR department?"
)
print(return_OAI(pqt8))


{
    "tables": {
        "employees": "Employee information",
    }
}
{
    "tables": {
        "employees": "Employee information",
        "studies": "Educational studies"
    }
}
{
    "tables": {
        "employees": "Employee information",
        "salary": "Salary details"
    }
}
{
    "tables": {
        "employees": "Employee information",
        "studies": "Educational studies"
    }
}
{
    "tables": {
        "employees": "Employee information",
        "studies": "Educational studies"
    }
}


Objective:

We tested whether GPT-3.5 Turbo could identify the correct SQL tables needed to answer a user question, based only on table names and short definitions.

Findings:

In most cases, GPT performed well and selected the appropriate tables accurately. For example, when asked about employees who received bonuses and had degrees, it correctly chose the salary and studies tables. It also gave reasonable short labels for each selected table, which matched the table definitions.

Weak Variations:

Some variations were less reliable:

Questions that involved indirect logic (e.g., "worked in more than one department") sometimes caused GPT to guess and include irrelevant tables like salary.

Abstract or vague questions, especially ones combining multiple timelines (e.g., graduation vs. hiring), sometimes resulted in incomplete or inconsistent table selection.

In rare cases, GPT hallucinated fields that weren't mentioned in any table description.

What I Learned:

-Clear, specific questions lead to better results.

-GPT struggles with vague or overly complex conditions unless the table descriptions are detailed.

-It's important to test multiple prompt phrasings to find the most effective structure.